In [1]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time

In [ ]:

BEARER_TOKEN = env.BEARER_TOKEN
ACCOUNT_ID = env.ID


In [3]:
def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,  #the filter是black box filter 01.json
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}",
            print(r.text)
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r

In [4]:
BATCH_ID = "9999"
# get filter from pre-written json file
f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json")) #之後這邊要補上遍歷各個filter的程式碼Z
all_cat_filter = json.load(f)
f.close()

# create black box query
print("create black box query...")
black_box_query_creation_response = create_black_box_query(all_cat_filter)  #response裡面是字典裝字典 第二個字典裡項目是key 值是資料內容




create black box query...
{
    "message": "Unauthorized",
    "errorCode": "rta-0",
    "errors": []
}


Exception: ('Error occured when creating black box query, status_code: 401', None)

In [ ]:
# parse black box query_id
print("parse black box query...")

black_box_query_id = ""
try:
    black_box_query_id = black_box_query_creation_response["data"]["id"]
except KeyError:
    raise (
        "No id found in response, error may occrued when creating black box query"
    )


In [ ]:
# get black box query result
print("get black box result")
black_box_query_id = black_box_query_creation_response["data"]["id"]
black_box_result_dict = get_black_box_query_result(
    black_box_query_id=black_box_query_id  #***********************
)

black_box_result_data = []
try:
    black_box_result_data = black_box_result_dict["data"]["tableData"]   #有一大堆商品，然後每個商品都有id跟asin。
except KeyError:
    raise (
        "Can't find table data, Error occrued when requesting for black box query result"
    )


In [ ]:
# create cerebro query for each asin in black_box_result_data   原本是product in black_box_result_data[:10]:
print("create cerebro query result")
cerebro_query_id_dict = {}
for product in black_box_result_data[:3]:
    asin = product["asin"]
    print(asin, end="..., ")
    r = create_cerebro_query(asin)
    print(r.status_code, end=", sleeping...\n")
    cerebro_query_id_dict[asin] = r 
    time.sleep(5)

In [ ]:
print("get cerebro query result")
cerebro_result_dict = {}
cerebro_query_id_list = [] ################ new
for asin, response in cerebro_query_id_dict.items():
    print(asin, end="\r")
    cerebro_query_id = ""
    
    try:
        cerebro_query_id = response.json()["data"]["id"]
        cerebro_query_id_list.append(cerebro_query_id) ############### new
    except KeyError:
        print("no result for this asin")
        continue

    cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
    print(cerebro_result_dict[asin].status_code, end="\r")
    time.sleep(5)

In [ ]:
# merging data
final_data = []      
for black_box_asin in cerebro_result_dict.keys() : 
    final_data = cerebro_result_dict[black_box_asin].json()['data']['tableData'].copy()
    for i in range(len(final_data)):
        final_data[i]['black_box_query_id'] =black_box_query_id #filter 01 的 black box id 
        final_data[i]['black_box_query_asin']=black_box_asin
        final_data[i]['cerebro_query_id']= cerebro_query_id_dict[black_box_asin].json()['data']['id']

In [ ]:
list =[1,2,3,4]
list2=list[3:4]
list2

0 10   10 20 150  157


[4]

In [ ]:
AIRTABLE_URL =env.AIRTABLE_URL
AIRTABLE_HEADER = {"Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"

}

In [ ]:

#format data to fit airtable api requiement
airtable_data = {'records':[]}
for i in range(len(final_data)):

    airtable_data['records'].append({'fields':{}
    })

    airtable_data['records'][i%10]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
    airtable_data['records'][i%10]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
    airtable_data['records'][i%10]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
    airtable_data['records'][i%10]['fields']['phraseId']=final_data[i]['phraseId']
    airtable_data['records'][i%10]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
    airtable_data['records'][i%10]['fields']['matchType']=final_data[i]['matchType'][0]##
    airtable_data['records'][i%10]['fields']['organicPosition']=final_data[i]['organicPosition']
    airtable_data['records'][i%10]['fields']['phrase']=final_data[i]['phrase']
    airtable_data['records'][i%10]['fields']['resultsNumber']=final_data[i]['resultsNumber']
    airtable_data['records'][i%10]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
    airtable_data['records'][i%10]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
    airtable_data['records'][i%10]['fields']['impressionExact30']=final_data[i]['impressionExact30']
    airtable_data['records'][i%10]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
    airtable_data['records'][i%10]['fields']['iq']=final_data[i]['iq']
    airtable_data['records'][i%10]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
    airtable_data['records'][i%10]['fields']['amazonChoice']=final_data[i]['amazonChoice']
    airtable_data['records'][i%10]['fields']['currentAsin']=final_data[i]['currentAsin']
    airtable_data['records'][i%10]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
    airtable_data['records'][i%10]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
    airtable_data['records'][i%10]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
    airtable_data['records'][i%10]['fields']['newCprExact']=final_data[i]['newCprExact']
    airtable_data['records'][i%10]['fields']['newCprBroad']=final_data[i]['newCprBroad']
    airtable_data['records'][i%10]['fields']['monthlySales']=final_data[i]['monthlySales']
    airtable_data['records'][i%10]['fields']['clickShareRate']=final_data[i]['clickShareRate']
    airtable_data['records'][i%10]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
    airtable_data['records'][i%10]['fields']['cpc']=final_data[i]['cpc']
    airtable_data['records'][i%10]['fields']['highCpc']=final_data[i]['highCpc']
    airtable_data['records'][i%10]['fields']['lowCpc']=final_data[i]['lowCpc']
    airtable_data['records'][i%10]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
    airtable_data['records'][i%10]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
    airtable_data['records'][i%10]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
    airtable_data['records'][i%10]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']

    
    if (i+1)%10 == 0 :
        r = requests.post(url=AIRTABLE_URL, headers=AIRTABLE_HEADER, json = airtable_data)
        if r.status_code == 200:
            print("success")
        else:
            raise Exception(
                f"Error occured when uploading, status_code: {r.status_code}",
                print(r.text)
    )
        airtable_data = {'records':[]}


success
success
success


KeyboardInterrupt: 

In [ ]:
r = requests.post(url=AIRTABLE_URL, headers=AIRTABLE_HEADER, json = airtable_data)
if r.status_code == 201:
    print("success")
else:
    raise Exception(
        f"Error occured when uploading, status_code: {r.status_code}",
        print(r.text)
    )

In [ ]:
airtable_data

{'records': [{'fields': {'black_box_query_id': 9348381,
    'phraseId': -214009248}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009249}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009250}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009251}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009252}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009253}},
  {'fields': {'black_box_query_id': 9348381, 'phraseId': -214009254}}]}

In [ ]:
len(final_data)

157